In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

In [2]:
import os
print(os.getcwd())

C:\Users\bzelasky\Documents\Python Scripts


In [3]:
def printDataFrameAttributes(dataframe, number):
    
    print('\nData shape:', dataframe.shape)
    print('\nType:', type(dataframe))
        
    print('\nData Types:\n')
    print(dataframe.dtypes)

    if (number > 0):
        print('\n print head \n')
        print(dataframe.head(number))
        print('\n')
        print('\n')
        print('\n print tail \n')
        print(dataframe.tail(number))
    
    return

def convertObjectToTimestamp(dataframe):

# this doesn't convert PM to 24 hour clock
#    dataframe['TimeStamp2']= pd.to_datetime(dataframe['TimeStamp']
                                            
    try:
        dataframe['TimeStamp2']= pd.to_datetime(dataframe['TimeStamp'], format='%m/%d/%Y %I:%M:%S.%f %p') #format='%m/%d/%Y %H:%M:%S.%f %p'
    except:        
        try:
            dataframe['TimeStamp2']= pd.to_datetime(dataframe['TimeStamp'], format='%m/%d/%Y %I:%M:%S %p')
        except:
            print('An error occurred - TimeStamp field is not in the format for the datetime formats tried.')
            return

## Read in data set for May

In [4]:
dataMay = pd.read_csv('VAV_data_may_20171115111552.txt', delimiter='\t')  
convertObjectToTimestamp(dataMay)

## Create DataSet
- How often is target temperature reached by 7 AM?
- Average time to reach target temperature for each VAV - by month, by week 
- Filter data - remove weekends, remove first floor
- Inpsect data - stats by VAV, floor, side (east/west), AC unit?


In [51]:
data_4 = dataMay.copy()

# set up some indexes so we can partition the data set by VAV and date
data_4['yearMonthDay'] = data_4['TimeStamp2'].map(lambda x:  pd.datetime(x.year, x.month, x.day))
data_4.set_index(['VAVCO', 'yearMonthDay', 'TimeStamp2'], inplace=True)

# create empty dataframe to append to
cols = (['Day','VAV', 'AC', 'Side', 'DayofWeek', 
         'Reading_At/After_3AM', 'RTemp_3AM', 
         'SP', 'Time_AtSP', 'RTemp_AtSP', '%Cool_AtSP', 'MinsToReachSP',
         'TIME_VAVStarted', 'RTemp_VAVStarted', 
         'Reading_At/After_8AM', 'RTemp_8AM', '%Cool_8AM',
         'cnt'])
StartTimeDF = pd.DataFrame(columns=cols)


# for each VAV flatten out the data set to create one row for each day in the month of may
totalRecordCount = 0  
for VAVCO in data_4.index.get_level_values('VAVCO').unique():   #Return an Index of values for requested level

     # select the data based on this index
     dfsub = data_4.loc[VAVCO]
     #printDataFrameAttributes(dfsub,2)
     
     # for each day in the month - iterate through saving off data at times of interes
     for day in dfsub.index.get_level_values('yearMonthDay').unique():            
            dfsub1 = dfsub.loc[day]
            
            # variables at 3AM or the first value after 3AM
            startValuesSet = False
            tempAtStart = np.nan
            timeAtStart = np.nan
            
            # variables when % cooling is > 0 for the first time
            vavStarted = False
            tempAtVavStarted = np.nan
            timeAtVavStarted = np.nan
            
            # variables when room temperature <= occupied set point
            occuppiedSetPointReached = False
            timeAtSetPoint = np.nan
            tempAtSetPoint = np.nan
            timeToSetPoint = np.nan
            vavOnAtSetPoint = False
            percentCoolingAtSetPoint = np.nan
            
            # variables at 8AM or the first value after 8AM - the end of the startup time
            endValueSet = False         
            tempAtEnd = np.nan
            timeAtEnd = np.nan
            percentCoolingAtEnd = np.nan
            
            totalRecordsPerDay = 0           
            evalDay = np.nan       
            
            for i, row in dfsub1.iterrows():  
                # common variables
                dayOfWeek = i.strftime("%A")
                evalDay = i.day 
                # variables to check we processed every recorde inthe dataframe
                totalRecordsPerDay = totalRecordsPerDay + 1
                totalRecordCount = totalRecordCount + 1 
                # start with values after 3AM
                if (i.hour >= 3):    
                    # capture values between 3AM & 8AM - not all days have values at or close to these times
                    # for now take the first value at or after these times - refine later
                    if (not startValuesSet) & (i.hour >= 3):     
                        startValuesSet = True
                        tempAtStart = row["Room Temperature"]
                        timeAtStart = i 
                    elif (not endValueSet) & (i.hour >= 8):
                        endValueSet = True
                        tempAtEnd = row["Room Temperature"]
                        timeAtEnd = i  
                        percentCoolingAtEnd = row["% cooling"]
                    # look to if room temperature ever hit setpoint
                     # capture data when % cooling > 0
                    if (not vavStarted) & (row["% cooling"] > 0):
                        vavStarted = True
                        tempAtVavStarted = row["Room Temperature"]
                        timeAtVavStarted = i
                    if ((not occuppiedSetPointReached) & 
                        (row["Room Temperature"] > row["Occupied Setpoint"] -1 ) &
                        (row["Room Temperature"] < row["Occupied Setpoint"] +1 )):
                        occuppiedSetPointReached = True
                        timeAtSetPoint = i
                        tempAtSetPoint = row["Room Temperature"]
                        if vavStarted:               
                            timeToSetPoint = (timeAtSetPoint - timeAtVavStarted).total_seconds()/60
                        else:
                            timeToSetPoint = (timeAtSetPoint - timeAtStart).total_seconds()/60
                        percentCoolingAtSetPoint = row["% cooling"]
                   
                            
            #print('endfor loop for row iteration', i.day)
            #print('\r tempAtStart', tempAtStart, ' timeAtStart', timeAtStart, ' timeAtSetPoint', timeAtSetPoint, ' tempAtSetPoint', tempAtSetPoint)
            
            # create dataframe to append 
            newdata = ([[evalDay, VAVCO, row['AC Unit'], row['Side'], dayOfWeek,
                         timeAtStart, tempAtStart,
                         row["Occupied Setpoint"], timeAtSetPoint, tempAtSetPoint, percentCoolingAtSetPoint, timeToSetPoint,
                         timeAtVavStarted, tempAtVavStarted, 
                         timeAtEnd, tempAtEnd, percentCoolingAtEnd,
                         totalRecordsPerDay]])
            df1 = pd.DataFrame(columns=cols, data = newdata)
            StartTimeDF = StartTimeDF.append(df1, ignore_index = True)
            
     #print('endfor loop for day level iteration for VAVCO: ', VAVCO)
    
#print('endfor loop for all VAVCOs')

print('*** Checks and balances ***')
print('\r Dataframe shape: ', dataMay.shape)
print('\r Total records processed:', totalRecordCount, '(should match # rows in data frame)')
print('\r\r *** Results ***')

StartTimeDF

*** Checks and balances ***
 Dataframe shape:  (75361, 13)
 Total records processed: 75361 (should match # rows in data frame)
 *** Results ***


,Day,VAV,AC,Side,DayofWeek,Reading_At/After_3AM,RTemp_3AM,SP,Time_AtSP,RTemp_AtSP,%Cool_AtSP,MinsToReachSP,TIME_VAVStarted,RTemp_VAVStarted,Reading_At/After_8AM,RTemp_8AM,%Cool_8AM,cnt
0,1,VAVCO 1-01,AC-1,NaN,Monday,2017-05-01 04:39:38.708618100,74.0,72.0,2017-05-01 10:19:43.427368100,72.5,12.10836,5.009375,2017-05-01 10:14:42.864868100,73.0,2017-05-01 10:14:42.864868100,73.0,19.591670,41
1,2,VAVCO 1-01,AC-1,NaN,Tuesday,2017-05-02 03:27:13.874954200,72.5,72.0,2017-05-02 03:27:13.874954200,72.5,0.00000,0.000000,2017-05-02 10:07:20.968704200,73.5,2017-05-02 08:47:11.953079200,73.5,0.000000,35
2,3,VAVCO 1-01,AC-1,NaN,Wednesday,2017-05-03 06:19:52.195343000,74.5,72.0,2017-05-03 18:55:03.617218,72.5,51.86623,515.139063,2017-05-03 10:19:55.273468,74.0,2017-05-03 09:49:48.648468000,74.0,0.000000,37
3,4,VAVCO 1-01,AC-1,NaN,Thursday,2017-05-04 04:33:06.804718000,74.5,72.0,2017-05-04 11:18:13.226593,72.5,29.06653,60.004427,2017-05-04 10:18:12.960968,73.5,2017-05-04 08:08:10.335968000,74.0,0.000000,24
4,5,VAVCO 1-01,AC-1,NaN,Friday,2017-05-05 07:35:02.648468000,73.0,72.0,2017-05-05 07:40:04.664093,72.5,0.00000,5.033594,NaN,NaN,2017-05-05 10:45:05.429718000,72.5,0.000000,21
5,6,VAVCO 1-01,AC-1,NaN,Saturday,2017-05-06 03:54:59.210968000,72.0,72.0,2017-05-06 03:54:59.210968,72.0,0.00000,0.000000,NaN,NaN,2017-05-06 08:31:59.585968000,71.5,0.000000,40
6,7,VAVCO 1-01,AC-1,NaN,Sunday,2017-05-07 04:56:56.804718000,72.5,72.0,2017-05-07 04:56:56.804718,72.5,0.00000,0.000000,NaN,NaN,2017-05-07 12:09:45.039093000,72.0,0.000000,15
7,8,VAVCO 1-01,AC-1,NaN,Monday,2017-05-08 07:11:03.896652200,72.5,72.0,2017-05-08 07:11:03.896652200,72.5,0.00000,0.000000,2017-05-08 10:06:06.974777200,72.0,2017-05-08 09:11:05.990402200,72.5,0.000000,21
8,9,VAVCO 1-01,AC-1,NaN,Tuesday,2017-05-09 03:11:20.853836000,73.0,72.0,2017-05-09 03:16:21.541336,72.5,0.00000,5.011458,2017-05-09 10:05:05.404724100,72.5,2017-05-09 10:00:04.826599100,72.0,0.000000,37
9,10,VAVCO 1-01,AC-1,NaN,Wednesday,2017-05-10 03:35:20.139099100,72.5,72.0,2017-05-10 03:35:20.139099100,72.5,0.00000,0.000000,NaN,NaN,2017-05-10 08:30:24.685974100,72.0,0.000000,32


## Data Understanding
* % set point was never reached
* % VAV unit was at set point at first reading

In [65]:
#sum(StartTimeDF[StartTimeDF.MinsToReachSP > 0 ])
sum(pd.isnull(StartTimeDF['MinsToReachSP']))

385

## Remove Data 
* First floor units
* Weekends
* Rows where setpoint was not reached e.g MinsToReachSP > -1

In [52]:
#printDataFrameAttributes(StartTimeDF, 0)
sub1= StartTimeDF[StartTimeDF.DayofWeek != 'Saturday']
sub1= sub1[sub1.DayofWeek != 'Sunday']
sub1= sub1[sub1.AC != 'AC-1']
#printDataFrameAttributes(sub1, 0)


In [53]:
sub1
print('\nData shape for sub1:', sub1.shape)
subSetPointReached= sub1[sub1.MinsToReachSP > -1]
print('\nData shape for subSetPointReached:', subSetPointReached.shape)


Data shape for sub1: (1863, 18)

Data shape for subSetPointReached: (1630, 18)


## Group Data & simple stats

In [55]:
# use groupby to get stats by VAV  
# https://www.shanelynn.ie/summarising-aggregation-and-grouping-data-in-python-pandas/
# if you calculate more than one column of results, your result will be a Dataframe. For a single column of results, the agg function, by default, will produce a Series.

#subSetPointReached.groupby(['VAV']).groups.keys()

# ?? how to get average time? not able to use in groupby below - needs to be numeric
print('Averages for month of May')
#out1 = subSetPointReached.groupby(['VAV', 'DayofWeek'])['RTemp_3AM', 'SP', 'MinsToReachSP', 'RTemp_AtSP', '%Cool_AtSP', 'RTemp_8AM', '%Cool_8AM'].mean()
out1 = subSetPointReached.groupby(['VAV', 'DayofWeek'])['RTemp_3AM', 'SP', 'MinsToReachSP', 'RTemp_AtSP', '%Cool_AtSP', 'RTemp_8AM', '%Cool_8AM'].agg([np.mean, np.std])

out1

#out1.describe() 

Averages for month of May


RTemp_3AM                   SP           MinsToReachSP  \
                           mean       std       mean       std          mean   
VAV        DayofWeek                                                           
VAVCO 2-01 Friday     72.833333  0.577350  72.000000  0.000000    218.391406   
           Monday     73.000000  0.866025  72.000000  0.000000    241.724502   
           Thursday   72.500000  0.500000  72.000000  0.000000      1.663455   
           Tuesday    72.250000  0.353553  72.000000  0.000000      0.000000   
           Wednesday  72.750000  0.645497  72.000000  0.000000    173.799835   
VAVCO 2-02 Friday     73.500000  0.500000  73.333333  1.154701     94.258049   
           Monday     75.000000  1.414214  73.000000  1.414214     27.504427   
           Thursday   73.625000  1.600781  73.500000  1.000000    368.063508   
           Tuesday    74.000000  1.354006  73.500000  1.000000     32.506510   
           Wednesday  73.600000  1.474788  72.800000  1.095445    168.037302   
VAVCO 2-03 Friday     72.833333  0.288675  72.666667  1.154701    369.018849   
           Monday     73.833333  1.527525  72.000000  0.000000    176.722625   
           Thursday   73.333333  1.527525  72.666667  1.154701     15.005816   
           Tuesday    73.125000  1.796988  72.000000  0.000000    235.577721   
           Wednesday  72.875000  1.547848  72.000000  0.000000     62.516146   
VAVCO 2-04 Friday     72.375000  0.478714  72.000000  0.000000      1.246875   
           Monday     72.000000  0.707107  72.000000  0.000000      0.000000   
           Thursday   72.625000  1.314978  72.000000  0.000000      5.000651   
           Tuesday    72.400000  1.673320  72.000000  0.000000    227.394091   
           Wednesday  72.700000  1.440486  72.000000  0.000000      5.999688   
VAVCO 2-05 Friday     73.125000  0.478714  72.250000  0.500000     95.023633   
           Monday     73.625000  1.314978  72.500000  1.000000    105.028558   
           Thursday   73.250000  1.258306  72.250000  0.500000     21.252865   
           Tuesday    73.250000  1.443376  72.250000  0.500000    164.746347   
           Wednesday  73.200000  1.151086  72.400000  0.547723     73.021510   
VAVCO 2-06 Friday     72.750000  0.288675  71.500000  2.516611     69.442625   
           Monday     72.500000  0.707107  72.000000  0.000000      2.494531   
           Thursday   72.875000  1.108678  72.500000  1.000000     65.015121   
           Tuesday    73.166667  1.755942  72.000000  0.000000    136.694097   
           Wednesday  73.100000  0.894427  72.800000  1.095445     64.015873   
...                         ...       ...        ...       ...           ...   
VAVCO 6-15 Friday     74.000000  0.408248  72.000000  0.000000     84.444618   
           Monday     74.100000  1.710263  72.000000  0.000000    118.034409   
           Thursday   74.375000  1.108678  72.000000  0.000000     61.729906   
           Tuesday    73.500000  1.732051  72.000000  0.000000     17.017187   
           Wednesday  74.100000  1.387444  72.000000  0.000000     24.371146   
VAVCO 6-16 Friday     73.166667  0.288675  70.666667  2.309401    104.314959   
           Monday     73.500000  1.500000  72.000000  0.000000    168.382869   
           Thursday   73.833333  1.040833  72.000000  0.000000    130.038009   
           Tuesday    73.500000  1.500000  72.000000  0.000000    128.377778   
           Wednesday  73.500000  1.224745  72.000000  0.000000     26.254167   
VAVCO 6-17 Friday     74.000000  0.500000  71.333333  3.055050     85.914004   
           Monday     74.000000  1.802776  73.333333  1.154701      5.004514   
           Thursday   74.666667  1.154701  72.666667  1.154701     23.503385   
           Tuesday    74.166667  1.258306  72.000000  2.000000      3.324566   
           Wednesday  74.000000  1.732051  72.500000  1.000000     44.221354   
VAVCO 6-18 Friday     73.750000  0.500000  72.000000  0.000000     78.195334   
           Monday     

In [66]:
sub1.describe()

# these two are the same
#subSetPointReached.describe()

,RTemp_3AM,SP,RTemp_AtSP,%Cool_AtSP,MinsToReachSP,RTemp_VAVStarted,RTemp_8AM,%Cool_8AM
count,1863.000000,1863.000000,1630.000000,1630.000000,1630.000000,1413.000000,1863.000000,1863.000000
mean,73.659689,72.144928,72.540798,29.048158,88.184720,73.861642,73.550993,16.784827
std,1.340969,1.062208,0.747017,33.751339,159.721245,1.271473,1.358261,30.149312
min,69.500000,68.000000,68.500000,0.000000,0.000000,69.500000,69.500000,0.000000
25%,72.500000,72.000000,72.500000,0.000000,0.000000,73.000000,72.500000,0.000000
50%,73.500000,72.000000,72.500000,13.525010,25.000301,73.500000,73.500000,0.000000
75%,74.500000,72.000000,72.500000,54.314077,80.009310,75.000000,74.500000,19.900000
max,79.000000,74.000000,74.500000,100.000000,1192.197975,79.000000,79.000000,100.000000
